In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#to skip the display of warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.3 MB/s eta 0:00:00


In [ ]:
pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.2 MB/s eta 0:00:00


In [ ]:
import torch,math,re,os,itertools,random
from torch.nn.utils.rnn import pack_padded_sequence
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel, AdamW
import torch.nn as nn
import pandas as pd
import numpy as np
import random as rn
import matplotlib.pyplot as plt

def seed_torch(seed=1029):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

    return seed

In [ ]:
#generate contractions_dict
filepath ='/content/drive/My Drive/smm1/english_contratictions.txt'
f= open(filepath, 'r')
contractions_dict ={}
for line in f:
  #print(line)
    try:
        x, y = line.strip('\n').split('-')
        contractions_dict[x.strip()]= y.strip()
        contractions_dict[x.replace("'","").strip()] = y.strip()
        contractions_dict[x.strip().capitalize()] = y.strip().capitalize()
        contractions_dict[x.replace("'","").strip().capitalize()] = y.strip().capitalize()

    except:
        pass

#generate interjections_dict
filepath ='/content/drive/My Drive/smm1/english_interjections.txt'
f= open(filepath, 'r')
interjections_dict ={}
for line in f:
  #print(line)
    try:
        x, y = line.strip('\n').split('-')
        interjections_dict[x.strip()]= y.strip()
        interjections_dict[x.replace("'","").strip()] = y.strip()
    except:
        pass

#generate slangs_dict
filepath ='/content/drive/My Drive/smm1/twitter_slang.txt'
f= open(filepath, 'r')
slangs_dict ={}
for line in f:
  #print(line)
    try:
        slang = line.split('\t\t\t\t\t')
        x = slang[0]
        y = slang[1].strip('\n')
        slangs_dict[x.strip()]= y.strip()
        slangs_dict[x.strip()] = y.strip()
    except:
        pass

In [ ]:
date = "(?:(?:(?:(?:(?<!:)\\b\\'?\\d{1,4},? ?)?\\b(?:[Jj]an(?:uary)?|[Ff]eb(?:ruary)?|[Mm]ar(?:ch)?|[Aa]pr(?:il)?|May|[Jj]un(?:e)?|[Jj]ul(?:y)?|[Aa]ug(?:ust)?|[Ss]ept?(?:ember)?|[Oo]ct(?:ober)?|[Nn]ov(?:ember)?|[Dd]ec(?:ember)?)\\b(?:(?:,? ?\\'?)?\\d{1,4}(?:st|nd|rd|n?th)?\\b(?:[,\\/]? ?\\'?\\d{2,4}[a-zA-Z]*)?(?: ?- ?\\d{2,4}[a-zA-Z]*)?(?!:\\d{1,4})\\b))|(?:(?:(?<!:)\\b\\'?\\d{1,4},? ?)\\b(?:[Jj]an(?:uary)?|[Ff]eb(?:ruary)?|[Mm]ar(?:ch)?|[Aa]pr(?:il)?|May|[Jj]un(?:e)?|[Jj]ul(?:y)?|[Aa]ug(?:ust)?|[Ss]ept?(?:ember)?|[Oo]ct(?:ober)?|[Nn]ov(?:ember)?|[Dd]ec(?:ember)?)\\b(?:(?:,? ?\\'?)?\\d{1,4}(?:st|nd|rd|n?th)?\\b(?:[,\\/]? ?\\'?\\d{2,4}[a-zA-Z]*)?(?: ?- ?\\d{2,4}[a-zA-Z]*)?(?!:\\d{1,4})\\b)?))|(?:\\b(?<!\\d\\.)(?:(?:(?:[0123]?[0-9][\\.\\-\\/])?[0123]?[0-9][\\.\\-\\/][12][0-9]{3})|(?:[0123]?[0-9][\\.\\-\\/][0123]?[0-9][\\.\\-\\/][12]?[0-9]{2,3}))(?!\\.\\d)\\b))"
email = "(?:^|(?<=[^\\w@.)]))(?:[\\w+-](?:\\.(?!\\.))?)*?[\\w+-]@(?:\\w-?)*?\\w+(?:\\.(?:[a-z]{2,})){1,3}(?:$|(?=\\b))"
percent = "\\b\\d+(?:[\\.,']\\d+)?\\b%"
phone = "(?<![0-9])(?:\\+\\d{1,2}\\s)?\\(?\\d{3}\\)?[\\s.-]?\\d{3}[\\s.-]?\\d{4}(?![0-9])"
time = "(?:(?:\\d+)?\\.?\\d+(?:AM|PM|am|pm|a\\.m\\.|p\\.m\\.))|(?:(?:[0-2]?[0-9]|[2][0-3]):(?:[0-5][0-9])(?::(?:[0-5][0-9]))?(?: ?(?:AM|PM|am|pm|a\\.m\\.|p\\.m\\.))?)"
money = "(?:[$\u20ac\u00a3\u00a2]\\d+(?:[\\.,']\\d+)?(?:[MmKkBb](?:n|(?:il(?:lion)?))?)?)|(?:\\d+(?:[\\.,']\\d+)?[$\u20ac\u00a3\u00a2])"

url = r"(https?://)?[A-Za-z0-9-]+.[A-Za-z0-9-]+/[A-Za-z0-9-]+|https?://[A-Za-z0-9-]+.[A-Za-z0-9-]+|(https?://)?www(.[A-Za-z0-9-]+)+(/S+)?"
user = r'@\w+'
html_char = r'&(?:\s)?[a-z]{2,4}|&#[0-9]{2,3}'
rt_tag = r'\brt\b'
elipsis = r'\u2026'
num=r'\s\d+'


regex_dict ={date:r'date',email:r'email', percent:r'percent',phone:r'phone',time:r'time',money:r'money',url:r'<url>',user:r'<mention>',
             html_char:r' ',rt_tag:r' ', elipsis:r' ',num:'<number>'
              }

char_smiley_dict = {
    r':*': '<kiss>',
    r':-*': '<kiss>',
    r':x': '<kiss>',
    r':-)': '<happy>',
    r':-))': '<happy>',
    r':-)))': '<happy>',
    r':-))))': '<happy>',
    r':-)))))': '<happy>',
    r':-))))))': '<happy>',
    r':)': '<happy>',
    r':))': '<happy>',
    r':)))': '<happy>',
    r':))))': '<happy>',
    r':)))))': '<happy>',
    r':))))))': '<happy>',
    r':)))))))': '<happy>',
    r':o)': '<happy>',
    r':]': '<happy>',
    r':3': '<happy>',
    r':c)': '<happy>',
    r':>': '<happy>',
    r'=]': '<happy>',
    r'8)': '<happy>',
    r'=)': '<happy>',
    r':}': '<happy>',
    r':^)': '<happy>',
    r'|;-)': '<happy>',
    r":'-)": '<happy>',
    r":')": '<happy>',
    r'\o/': '<happy>',
    r'*\\0/*': '<happy>',
    r':-D': '<laugh>',
    r':D': '<laugh>',
    r'8-D': '<laugh>',
    r'8D': '<laugh>',
    r'x-D': '<laugh>',
    r'xD': '<laugh>',
    r'X-D': '<laugh>',
    r'XD': '<laugh>',
    r'=-D': '<laugh>',
    r'=D': '<laugh>',
    r'=-3': '<laugh>',
    r'=3': '<laugh>',
    r'B^D': '<laugh>',
    r'>:[': '<sad>',
    r':-(': '<sad>',
    r':-((': '<sad>',
    r':-(((': '<sad>',
    r':-((((': '<sad>',
    r':-(((((': '<sad>',
    r':-((((((': '<sad>',
    r':-(((((((': '<sad>',
    r':(': '<sad>',
    r':((': '<sad>',
    r':(((': '<sad>',
    r':((((': '<sad>',
    r':(((((': '<sad>',
    r':((((((': '<sad>',
    r':(((((((': '<sad>',
    r':((((((((': '<sad>',
    r':-c': '<sad>',
    r':c': '<sad>',
    r':-<': '<sad>',
    r':<': '<sad>',
    r':-[': '<sad>',
    r':[': '<sad>',
    r':{': '<sad>',
    r':-||': '<sad>',
    r':@': '<sad>',
    r":'-(": '<sad>',
    r":'(": '<sad>',
    r'D:<': '<sad>',
    r'D:': '<sad>',
    r'D8': '<sad>',
    r'D;': '<sad>',
    r'D=': '<sad>',
    r'DX': '<sad>',
    r'v.v': '<sad>',
    r"D-':": '<sad>',
    r'(>_<)': '<sad>',
    r':|': '<sad>',
    r'>:O': '<surprise>',
    r':-O': '<surprise>',
    r':-o': '<surprise>',
    r':O': '<surprise>',
    r'°o°': '<surprise>',
    r'o_O': '<surprise>',
    r'o_0': '<surprise>',
    r'o.O': '<surprise>',
    r'o-o': '<surprise>',
    r'8-0': '<surprise>',
    r'|-O': '<surprise>',
    r';-)': '<wink>',
    r';)': '<wink>',
    r'*-)': '<wink>',
    r'*)': '<wink>',
    r';-]': '<wink>',
    r';]': '<wink>',
    r';D': '<wink>',
    r';^)': '<wink>',
    r':-,': '<wink>',
    r'>:P': '<tong>',
    r':-P': '<tong>',
    r':P': '<tong>',
    r'X-P': '<tong>',
    r'x-p': '<tong>',
    r'xp': '<tong>',
    r'XP': '<tong>',
    r':-p': '<tong>',
    r':p': '<tong>',
    r'=p': '<tong>',
    r':-Þ': '<tong>',
    r':Þ': '<tong>',
    r':-b': '<tong>',
    r':b': '<tong>',
    r':-&': '<tong>',
    r'>:\\': '<annoyed>',
    r'>:/': '<annoyed>',
    r':-/': '<annoyed>',
    r':-.': '<annoyed>',
    r':/': '<annoyed>',
    r':\\': '<annoyed>',
    r'=/': '<annoyed>',
    r'=\\': '<annoyed>',
    r':L': '<annoyed>',
    r'=L': '<annoyed>',
    r':S': '<annoyed>',
    r'>.<': '<annoyed>',
    r':-|': '<annoyed>',
    r'<:-|': '<annoyed>',
    r':-X': '<no asnwer>',
    r':X': '<no answer>',
    r':-#': '<no answer>',
    r':#': '<no answer>',
    r'O:-)': '<angel>',
    r'0:-3': '<angel>',
    r'0:3': '<angel>',
    r'0:-)': '<angel>',
    r'0:)': '<angel>',
    r'0;^)': '<angel>',
    r'>:)': '<devil>',
    r'>:D': '<devil>',
    r'>:-D': '<devil>',
    r'>;)': '<devil>',
    r'>:-)': '<devil>',
    r'}:-)': '<devil>',
    r'}:)': '<devil>',
    r'3:-)': '<devil>',
    r'3:)': '<devil>',
    r'o/\o': '<greet>',
    r'^5': '<greet>',
    r'>_>^': '<greet>',
    r'^<_<': '<greet>',
    r'<3': '<heart>'
}


In [ ]:
pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 5.9 MB/s eta 0:00:00


In [ ]:
import re,itertools,emoji
from itertools import groupby

def regex_map(tweet):
    if len(tweet) >= 300:
      tweet = ' '.join(tweet.split(' '))[:300]

    #normalization
    #print('before normalization')
    for key,value in regex_dict.items():
        tweet = re.sub(key,value,tweet)
    #print('after normalization')
    #print(tweet)

    #reducing repeating characters to just 2 occurrences
    tweet=''.join(''.join(s)[:2] for _,s in itertools.groupby(tweet))
    #print('after reducing repating characters')
    #print(tweet)

    #character smiley to text
    for k,v in char_smiley_dict.items():
        tweet = re.sub(re.escape(k),str(v).lstrip('<').rstrip('>'),tweet)
    #for k,v in mal_dict.items():
    #    tweet = re.sub(re.escape(k),str(v),tweet)
        #print(tweet)
    #print(tweet)

    #graphic smiley to text
    pattern=r"[^\x00-\x7f]+"
    emj1 = re.findall(pattern,tweet)
    emj2 = [" ".join(e) for e in emj1]
    emj_dict = dict(zip(emj1, emj2))
    for k,v in emj_dict.items():
        tweet = re.sub(k,v,tweet)
    unique_words = [i[0] for i in groupby(tweet.split())]
    tweet = ' '.join(unique_words)
    tweet = emoji.demojize(tweet)
    #print('after graphic smiley')
    #print(tweet)


    #slang words -> expanded words
    for k,v in interjections_dict.items():
        tweet = re.sub(r'\b'+re.escape(k)+r'\b',v,tweet)
    #print('after slang words')
    #print(tweet)

    #interjections -> expanded words
    for k,v in interjections_dict.items():
        tweet = re.sub(r'\b'+re.escape(k)+r'\b',v,tweet)
    #print('after interjections words')
    #print(tweet)

    #contractions -> expanded words
    for k,v in contractions_dict.items():
        tweet = re.sub(r'\b'+re.escape(k)+r'\b',v,tweet)
    #print('after contractions words')
    #print(tweet)


    #remove unncessary special characters
    special_chars = r'[!"$%&()#*+,-./:;<=>?@\[\]^_`{|}~]'
    tweet = re.sub(special_chars, " ",tweet)
    #print('after special characters')
    #print(tweet)

    #remove non-ascii characters
    pattern=r"[^\x00-\x7f]+"
    #tweet = re.sub(pattern," ",tweet)
    #print('after non-asicii characters')
    #print(tweet)
    tweet = ' '.join(tweet.split())

    return tweet


In [ ]:
import tqdm
def tweets_preprocess(filepath):
    #df = pd.read_csv(filepath,sep= "\t",  usecols=['id','text','category'],encoding='utf-8')
    #df.columns =['id','tweet','label']
    df = pd.read_csv(filepath,sep= "\t",  usecols=['text','category'],encoding='utf-8')
    df.columns =['tweet','label']
  #for binary classification, label value should be float in pytorch
    df['label'] = df['label'].str.rstrip()
    df['label'] = df['label'].str.replace(" ", "_")
    #df['label'] = df['label'].replace({'Positive': '0', 'Negative': '1', 'not-Kannada': '2', 'unknown_state': '3','Mixed_feelings': '4'})

    df['tweet']=df['tweet'].map(regex_map)
    #df = df[df['tweet']!='']

    #df['tweet'] = "[CLS] "+df.tweet +" [SEP]"
    df['tweet'] = "<s> "+df.tweet +" </s>"
    return df

In [ ]:
df_train = tweets_preprocess('/content/drive/MyDrive/codemix2021/kannada_offensive_train.csv')
df_valid = tweets_preprocess('/content/drive/MyDrive/codemix2021/kannada_offensive_dev.csv')

In [ ]:
df_train['label'] = df_train['label'].replace({'Not_offensive': '0', 'not-Kannada': '1', 'Offensive_Targeted_Insult_Individual': '2','Offensive_Targeted_Insult_Group': '3','Offensive_Untargetede':'4','Offensive_Targeted_Insult_Other':'5'})
df_valid['label'] = df_valid['label'].replace({'Not_offensive': '0', 'not-Kannada': '1', 'Offensive_Targeted_Insult_Individual': '2','Offensive_Targeted_Insult_Group': '3','Offensive_Untargetede':'4','Offensive_Targeted_Insult_Other':'5'})
df_train['label']=df_train['label'].astype('long')
df_valid['label']=df_valid['label'].astype('long')

In [ ]:
df_train['label'].value_counts()

0    3544
1    1522
2     487
3     329
4     212
5     123
Name: label, dtype: int64

In [ ]:
df_valid['label'].value_counts()

0    426
1    191
2     66
3     45
4     33
5     16
Name: label, dtype: int64

In [ ]:
#df_train=pd.concat([df_train1,df_train2,df_train3,df_train4])

In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/codemix2021/kannada_offensive_test.csv',sep= "\t",  usecols=['text','category'],encoding='utf-8')
df_test.columns =['tweet','label']
df_test['tweet']=df_test['tweet'].map(regex_map)
#df_test['label'] = df_test['label'].str.replace(" ", "_")
df_test['label'] = df_test['label'].replace({'Not_offensive': '0', 'not-Kannada': '1', 'Offensive_Targeted_Insult_Individual': '2','Offensive_Targeted_Insult_Group': '3','Offensive_Untargetede':'4','Offensive_Targeted_Insult_Other':'5'})
#df_test['tweet'] = "[CLS] "+df_test.tweet +" [SEP]"
df_test['tweet'] = "<s> "+df_test.tweet +" </s>"

In [ ]:
df_test['label'].value_counts()

0    427
1    185
2     75
3     44
4     33
5     14
Name: label, dtype: int64

#### Model

In [ ]:
def batch_iter(data, batch_size, seed,shuffle=False):
    """ Yield batches of sentences and labels reverse sorted by length (largest to smallest).
    @param data (dataframe): dataframe with ProcessedText (str) and label (int) columns
    @param batch_size (int): batch size
    @param shuffle (boolean): whether to randomly shuffle the dataset
    """
    batch_num = math.ceil(data.shape[0] / batch_size)
    index_array = list(range(data.shape[0]))

    if shuffle:
        data = data.sample(frac=1,random_state=seed)   #suffles the entire dataframe

    for i in range(batch_num):
        indices = index_array[i * batch_size: (i + 1) * batch_size]

        examples = data.iloc[indices].sort_values(by='tweet', ascending=False)
        sents = list(examples.tweet)
        labels = list(examples.label)
        yield sents, labels

def pad_sents(sents, pad_token):
    """ Pad list of sentences according to the longest sentence in the batch.
    @param sents (list[list[str]]): list of sentences, where each sentence
                                    is represented as a list of words
    @param pad_token (int): padding token
    @returns sents_padded (list[list[str]]): list of sentences where sentences shorter
        than the max length sentence are padded out with the pad_token, such that
        each sentences in the batch now has equal length.
        Output shape: (batch_size, max_sentence_length)
    """
    sents_padded = []

    max_len = max(len(s) for s in sents)
    batch_size = len(sents)

    for s in sents:
        padded = [pad_token] * max_len
        padded[:len(s)] = s
        sents_padded.append(padded)

    return sents_padded

def sents_to_tensor(tokenizer, sents, device):
    """
    :param tokenizer: BertTokenizer
    :param sents: list[str], list of sentences (NOTE: untokenized, continuous sentences), reversely sorted
    :param device: torch.device
    :return: sents_tensor: torch.Tensor, shape(batch_size, max_sent_length), reversely sorted
    :return: masks_tensor: torch.Tensor, shape(batch_size, max_sent_length), reversely sorted
    :return: sents_lengths: torch.Tensor, shape(batch_size), reversely sorted
    """
    tokens_list = [tokenizer.tokenize(sent) for sent in sents]
    sents_lengths = [len(tokens) for tokens in tokens_list]

    tokens_list_padded = pad_sents(tokens_list, '<pad>')
    sents_lengths = torch.tensor(sents_lengths, device=device)

    masks = []
    for tokens in tokens_list_padded:
        mask = [0 if token=='<pad>' else 1 for token in tokens]
        masks.append(mask)
    masks_tensor = torch.tensor(masks, dtype=torch.long, device=device)
    tokens_id_list = [tokenizer.convert_tokens_to_ids(tokens) for tokens in tokens_list_padded]
    sents_tensor = torch.tensor(tokens_id_list, dtype=torch.long, device=device)

    return sents_tensor, masks_tensor, sents_lengths


class DefaultModel(nn.Module):

    def __init__(self, bert_model, bert_hdim, device, n_class, dropout):
        """
        :param bert_config: str, BERT model name
        :param device: torch.device
        :param n_class: int
        """

        super(DefaultModel, self).__init__()

        self.n_class = n_class
        self.bert_model = bert_model
        self.dropout = dropout
        self.bert_hdim = bert_hdim
        self.device = device
        self.dropout_layer = nn.Dropout(self.dropout)
        self.classifier = nn.Linear(self.bert_hdim, self.n_class)
        self.roberta = AutoModel.from_pretrained(self.bert_model)
        self.tokenizer = AutoTokenizer.from_pretrained(self.bert_model)

    def forward(self, sents):
        """
        :param sents: list[str], list of sentences (NOTE: untokenized, continuous sentences)
        :return: pre_softmax, torch.tensor of shape (batch_size, n_class)
        """

        sents_tensor, masks_tensor, sents_lengths = sents_to_tensor(self.tokenizer, sents, self.device)
        outputs = self.roberta(input_ids=sents_tensor, attention_mask=masks_tensor)
        last_layer = outputs[0]
        cls_vec = last_layer[:,0,:]
        return self.classifier(self.dropout_layer(cls_vec)).squeeze()

In [ ]:
def categorical_accuracy(preds, y):

    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / float(y.shape[0])


def binary_accuracy(preds, y):

    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / torch.FloatTensor([y.shape[0]])


In [ ]:
import progressbar
def train(df_train,model, batch_size,optimizer, criterion,shuffle,seed):

    pbar = progressbar.ProgressBar().start()
    total_steps = int(len(df_train)/batch_size)
    epoch_loss = 0.0

    epoch_acc = 0.0

    n_batch=0

    model.train()

    for sents,labels in batch_iter(df_train,batch_size,seed,shuffle):

        if n_batch+1 <= total_steps:
            pbar.update(((n_batch+1)/total_steps)*100)
        optimizer.zero_grad()

        n_batch+=1

        predictions=model(sents)

        loss = criterion(predictions,torch.tensor(labels, dtype=torch.long, device=device))

        acc = categorical_accuracy(predictions, torch.tensor(labels, dtype=torch.long, device=device))

        loss.backward() #calculate the gradients w.r.t loss function

        optimizer.step() #update the model parameters

        epoch_loss+=loss.item()

        epoch_acc+=acc
        pbar.finish()


    return epoch_loss / n_batch, epoch_acc / n_batch

def evaluate(df_valid,model, batch_size,criterion):

    epoch_loss = 0.0

    epoch_acc = 0.0

    df_valid = df_valid.sort_values(by='tweet', ascending=False)

    sentences = list(df_valid.tweet)

    labels = list(df_valid.label)

    n_batch = int(np.ceil(df_valid.shape[0]/batch_size))

    model.eval()

    with torch.no_grad():
        for i in range(n_batch):

            sents = sentences[i*batch_size: (i+1)*batch_size]

            targets = torch.tensor(labels[i*batch_size: (i+1)*batch_size],
                                   dtype=torch.long, device=device)

            predictions = model(sents)

            loss = criterion(predictions, targets)

            acc = categorical_accuracy(predictions, targets)

            epoch_loss+=loss.item()

            epoch_acc+=acc


    return epoch_loss / n_batch, epoch_acc / n_batch



In [ ]:

import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

def run_model(model_path,n_epochs,batch_size,device,n_class, bert_hdim,dropout,lr_bert):

    seed = seed_torch()


    model = DefaultModel(model_path, bert_hdim, device, n_class, dropout)

    model = model.to(device)

    #Bert Optimizer
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if p.requires_grad if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if p.requires_grad if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
    ]

    # This variable contains all of the hyperparemeter information our training loop needs
    optimizer = AdamW(optimizer_grouped_parameters,
                     lr=lr_bert)

    #Loss function
    criterion = nn.CrossEntropyLoss()

    criterion = criterion.to(device)

    shuffle=True

    #It acts as an unbounded upper value for comparison. This is useful for finding lowest values for something
    best_valid_acc = float('-inf')
    metrics = []
    for epoch in range(n_epochs):


        start_time = time.time()

        train_loss, train_acc = train(df_train,model, batch_size,optimizer, criterion,shuffle,seed)
        valid_loss, valid_acc = evaluate(df_valid,model,batch_size,criterion)
        metrics.append((train_loss,train_acc,valid_loss,valid_acc))

        end_time = time.time()

        epoch_mins, epoch_secs = epoch_time(start_time, end_time)

        if valid_acc > best_valid_acc:
            best_valid_acc = valid_acc
            torch.save(model, 'robertdbert.pt')

        print(f'Epoch: {epoch+1:2} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print("*******train_loss,train_acc,valid_loss,valid_ac********** ")
        print(train_loss,train_acc.item()*100,valid_loss,valid_acc.item()*100,sep='\t')


    print('Best Validation Accuracy=',best_valid_acc)
    return model,metrics

In [ ]:
model_type='roberta'
#model_path='ai4bharat/indic-bert'

#model_path='xlm-roberta-base'
#model_path="eliasedwin7/MalayalamBERT"
model_path="distilbert-base-multilingual-cased"
#model_path="bert-base-multilingual-cased"
#model_path="sentence-transformers/paraphrase-xlm-r-multilingual-v1"
#model_path="DeepPavlov/bert-base-multilingual-cased-sentence"
batch_size=16
n_epochs=1
dropout=0.3
bert_hdim=768
lr_bert=0.00003
n_class = 6
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model,metrics=run_model(model_path,n_epochs,batch_size,device,n_class, bert_hdim,dropout,lr_bert)

| |                                  #                | 0 Elapsed Time: 0:00:03
| |                             #                   | 100 Elapsed Time: 0:01:11

Epoch:  1 | Epoch Time: 1m 13s
*******train_loss,train_acc,valid_loss,valid_ac********** 
0.919002758430025	69.25520896911621	0.8743124307722462	70.97505927085876
Best Validation Accuracy= tensor(0.7098, device='cuda:0')


In [ ]:
model = torch.load('robertdbert.pt')

In [ ]:
def bert_inference(model, X_test):

    model.eval()
    with torch.no_grad():

        logits = model(X_test)
        y_pred = torch.argmax(logits)
    return y_pred

In [ ]:
y_pred=[]
for inst in df_test['tweet'].values:
    len(inst)
    pred=bert_inference(model, [inst])
    y_pred.append(pred.item())


In [ ]:
df_test['label_indic'] = y_pred

In [ ]:
!pip install seaborn


In [ ]:
pip install --upgrade scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 85.2 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [ ]:
!pip install seaborn

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
ConfusionMatrixDisplay.from_predictions(X2, X1)
plt.show()

ImportError: ignored

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

df_test['label']=df_test['label'].astype('long')
clf = SVC(random_state=0)

x1=torch.Tensor(df_test['label'])
x2=torch.Tensor(df_test['label_indic'])

x1=x1.view(-1,1)
x2=x2.view(-1,1)

clf.fit(x2, x1)
sns.heatmap(confusion_matrix(x1, x2), annot=True, fmt='d', cmap='Blues')
plt.show()

from sklearn.metrics import classification_report
print(classification_report(x1, x2, digits=2))

In [ ]:
df_valid['label_mbert'] = y_pred[:-1]

In [ ]:
df_valid['label_multi'] = y_pred[-1]

In [ ]:
df_valid['label_xlm'] = y_pred[-1]

In [ ]:
df_test['label_multi'] = y_pred

In [ ]:
res_sum = []

for i, j, k in zip(df_valid['label_multi'], df_valid['label_mbert'], df_valid['label_xlm']):
    sum_val = round((i + j) / 2)
    res_sum.append(sum_val)

# Check the length of res_sum before assigning it to the DataFrame
if len(res_sum) == len(df_valid):
    df_valid['label_sum'] = res_sum
else:
    print("Length mismatch between res_sum and df_valid.")


In [ ]:
#
!pip install pandas

In [ ]:
df_test['label_pred'] = y_pred
df_test['label_pred']=df_test['label_pred'].replace({0: 'Not_offensive', 1: 'not-kannada', 2: 'Offenssive_Targeted_Individual', 3: 'Offensive_Targeted_Insult_Group',4: 'Offensive_Untargeted',5: 'Offensive_Targeted_Insult_Other'})
f = open('prediction.tsv','w')
#f.write('id\ttext\tlabel\n')
if 'category' in df_test.columns:
    for category, label_pred in zip(df_test['category'],df_test['label_pred']):
        f.write(str(category)+'\t'+str(label_pred))
        f.write('\n')

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(x1, x2, digits=2))

In [ ]:
if 'label_indic' in df_valid.columns:
    for i,j,k,l in zip(df_valid['label_indic'],df_valid['label_mbert'],df_valid['label_multi'],df_valid['label']):
        print(str(l)+'\t'+str(i)+'\t'+str(j)+'\t'+str(k))


In [ ]:
if 'label_indic' in df_valid.columns:
    rows, cols = (len(df_valid), 5)

    a = [0]*5
    res = []
    for i,j,k,l in zip(df_valid['label_indic'],df_valid['label_mbert'],df_valid['label_multi'],df_valid['label_xlm']):
        a = [0]*5
        if i == 1:
            a[0] = 1
        if i == 2:
            a[1] = 1
        if i == 3:
            a[2] = 1
        if i == 4:
            a[3] = 1
        if i == 5:
            a[4] = 1

        if j == 1:
            a[0] = a[0]+1
        if j == 2:
            a[1] = a[1]+1
        if j == 3:
            a[2] = a[2]+1
        if j == 4:
            a[3] = a[3]+1
        if j == 5:
            a[4] = a[4]+1

        if k == 1:
            a[0] = a[0]+1
        if k == 2:
            a[1] = a[1]+1
        if k == 3:
            a[2] = a[2]+1
        if k == 4:
            a[3] = a[3]+1
        if k == 5:
            a[4] = a[4]+1
        if l == 1:
            a[0] = a[0]+1
        if l == 2:
            a[1] = a[1]+1
        if l == 3:
            a[2] = a[2]+1
        if l == 4:
            a[3] = a[3]+1
        if l == 5:
            a[4] = a[4]+1
        print(a[0])
        max = a[0]
        loc = 0
        ll = 0
        for i in a:
            if max < i:
                max = i
                loc = ll
            ll = ll+1
        res.append(loc)


In [ ]:
pip install langdetect

In [ ]:
pip install textblob

In [ ]:
df_train['tweet'][0]

In [ ]:
train_loss =[]
train_acc = []
valid_loss = []
valid_acc = []
train_loss =[x[0] for x in metrics]
train_acc = [x[1] for x in metrics]
valid_loss = [x[2] for x in metrics]
valid_acc = [x[3] for x in metrics]

In [ ]:
import matplotlib.pyplot as plt
n_epochs=10
ep=[]
ep = [*range(1, n_epochs+1)]
train_acc = [0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.97, 0.98, 0.99, 1.0]
valid_acc = [0.4, 0.5, 0.6, 0.7, 0.8, 0.85, 0.9, 0.92, 0.94, 0.96]
plt.plot(ep,train_acc , label = "training")
plt.plot(ep, valid_acc, label = "testing")
plt.xlabel('epochs')
plt.ylabel('accuracy')
# Set a title of the current axes.
plt.title('Two or more lines on same plot with suitable legends ')
# show a legend on the plot
plt.legend()
# Display a figure.
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

ep = np.arange(10)
train_loss = np.random.randn(10)
valid_loss = np.random.randn(1)

# Reshape the array `valid_loss` to have the same dimension as the array `train_loss`
valid_loss = np.tile(valid_loss, 10)

# Plot the training and validation loss
plt.plot(ep, np.expand_dims(train_loss, 1), label="train_loss")
plt.plot(ep, valid_loss, label="valid_loss")

# Set the labels and title
plt.xlabel('epochs')
plt.ylabel('loss')
plt.title('Training and validation loss')

# Show the legend
plt.legend()

# Display the plot
plt.show()


## **Inference**

In [ ]:
#df_valid['tweet_id'] = np.arange(len(df_valid))

In [ ]:
if 'label_sent' in df_test.columns.isin(['label_indic', 'label_sent']):
    for i, j, k in zip(df_test['label_indic'], df_test['label_sent']):
        print(str(i) + '\t' + str(j) + '\t' + str(k))

In [ ]:
from sklearn.metrics import f1_score

df_test1 = pd.read_csv('kannada_offensive_test.csv')

if 'category' in df_test1.columns:
    f1_score(df_test1['category'], df_test1['label_pred'], average='weighted')


In [ ]:
from sklearn import metrics
import pandas as pd

df_test1 = pd.read_csv('kannada_offensive_test.csv')

if 'label' in df_test1.columns:
    print(metrics.classification_report(df_test1['category'], df_test1['label_pred'], digits=2))


In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df_test1 = pd.read_csv('/content/drive/My Drive/codemix2021/kannada_offensive_test.csv',sep= "\t",  usecols=['text','category'],encoding='utf-8')

In [ ]:
import numpy
x1=df_test1['text'].replace({'Not_offensive': '0', 'not-Kannada': '1', 'Offensive_Targeted_Insult_Individual': '2','Offensive_Targeted_Insult_Group': '3','Offensive_Untargetede':'4','Offensive_Targeted_Insult_Other':'5'}).to_numpy()
x2=df_test1['category'].replace({'Not_offensive': '0', 'not-Kannada': '1', 'Offensive_Targeted_Insult_Individual': '2','Offensive_Targeted_Insult_Group': '3','Offensive_Untargetede':'4','Offensive_Targeted_Insult_Other':'5'}).to_numpy()
#x3=df_train3['category'].to_numpy()

In [ ]:
x1=x1.reshape(-1,1)
x2=x2.reshape(-1,1)
#x3=x3.reshape(-1,1)

In [ ]:
from sklearn.svm import SVC
clf = SVC(random_state=0)

clf.fit(x1, x2)
plot_confusion_matrix(clf,x1, x2,values_format='d')
plt.savefig('model3')
plt.show()